In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam

In [ ]:
# Load the dataset
df1 = pd.read_csv('lotto_polonia_3.csv', delimiter=';')
df1.head()

In [ ]:
df1.tail()

In [ ]:
df = df1.iloc[:-1]
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df1.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Drop unnecessary columns
df.drop(['Numer', 'Dzien', 'Miesiac', 'Rok'], axis=1, inplace=True)
df.head()

In [ ]:
# df = df.iloc[4423:]
# df

In [ ]:
# Standardize the dataset
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [ ]:
# Prepare input and output data
number_of_rows = df.values.shape[0]
window_length = 21
number_of_features = df.values.shape[1]
X = np.empty([len(transformed_dataset) - window_length, window_length, number_of_features], dtype=float)
y = np.empty([len(transformed_dataset) - window_length, number_of_features], dtype=float)

for i in range(0, len(transformed_dataset) - window_length):
    X[i] = transformed_dataset[i : i + window_length]
    y[i] = transformed_dataset[i + window_length]

In [ ]:
# Split the data into training and testing sets
train_test_split = int(0.8 * len(X))
X_train, X_test = X[:train_test_split], X[train_test_split:]
y_train, y_test = y[:train_test_split], y[train_test_split:]

In [ ]:
# Define the model architecture
model = Sequential()
model.add(Conv1D(filters=500, kernel_size=3, activation='relu', input_shape=(window_length, number_of_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
model.add(Conv1D(filters=500, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.3))
model.add(Conv1D(filters=500, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(300, activation='relu'))
model.add(Dense(number_of_features, activation='linear'))

In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.002), loss='mse', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(x=X_train, y=y_train, batch_size=100, epochs=150, verbose=2, validation_data=(X_test, y_test))

In [ ]:
# Make a prediction
to_predict = df.tail(21)
scaled_to_predict = scaler.transform(to_predict.values)
y_pred = model.predict(np.array([scaled_to_predict]))
print('The predicted numbers in the last lottery game are:', scaler.inverse_transform(y_pred).astype(int)[0])

In [ ]:
# numbers1 = np.array([1,9,11,13,22,23,25,31,33,42,45,51,57,59,60,63,65,70,74,77])
#[ 3  7 11 15 19 23 27 30 34 38 42 46 50 54 57 61 65 69 73 77]
numbers1 = np.array([2,4,6,10,17,23,24,26,30,44,45,50,53,58,61,66,75,78,79,80])

In [ ]:
np.intersect1d(scaler.inverse_transform(y_pred).astype(int)[0], numbers1)

In [ ]:
# Inverse transform the predicted values to get the original scale
predicted_numbers = scaler.inverse_transform(y_pred).astype(int)[0]

In [ ]:
# Calculate the probability of each predicted number
probabilities = []
for number in predicted_numbers:
    occurrences = df.tail(20)[df.tail(20) == number].count().sum()  # Count the number of times the predicted number has occurred in the next 20 numbers
    probability = occurrences / len(df.tail(20)) * 100  # Calculate the probability as a percentage among the next 20 numbers
    probabilities.append(probability)

In [ ]:
# Normalize the probabilities so that they add up to 100%
probabilities = [p/sum(probabilities)*100 for p in probabilities]

In [ ]:
# Print the predicted numbers and their corresponding probabilities
for i in range(len(predicted_numbers)):
    print(f"The predicted number {predicted_numbers[i]} has a probability of {probabilities[i]:.2f}% to be drawn next.")

In [ ]:
x = np.array([6, 9, 14, 17, 19, 22, 27, 31, 35, 36, 42, 48, 49, 48, 54, 60, 62, 66, 71, 75])

y = np.intersect1d(scaler.inverse_transform(y_pred).astype(int)[0], x)
print(y)

np.intersect1d(numbers1, y)